In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import glob
import itertools
import pickle
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import save_image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from sklearn.model_selection import train_test_split

from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from collections import OrderedDict

In [ ]:
# Root directory for project
proj_root = ""

# Root directory for dataset
data_root = "/datasets/home/73/673/h6gupta/Project/Sematic_Segmentation/cityscapes/"

# Number of images in the directory
num_images = 2975

# Batch size during training
batch_size = 16

# Spatial size of training images. All images will be resized to this size using a transformer.
image_size = 256

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of feature maps in generator
ngf = 64

# Number of training epochs
num_epochs = 100

# Number of GPUs available. Use 0 for CPU mode.
ngpu = torch.cuda.device_count()

# Device to run on
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
class DataLoader:
    def __init__(self):
        '''
        Parameters:
        
        '''
        self.device = device
        self.data_path = data_root
        self.image_size = image_size
        self.batch_size = batch_size
        self.train_names = glob.glob(self.data_path + 'trainA/*')
        self.names = [self.train_names[i].split('/')[-1] for i in range(len(self.train_names))]
        self.data_transforms = torchvision.transforms.Compose([
                torchvision.transforms.Resize(image_size),
                torchvision.transforms.CenterCrop(image_size),
                torchvision.transforms.ToTensor(),
#                 torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
    
    def image_loader(self, image_name):
        """load image, returns cuda tensor"""
        image = Image.open(image_name)
        image = self.data_transforms(image).float()
        image = torch.autograd.Variable(image, requires_grad=False)
        image = image.unsqueeze(0)  # this is for VGG, may not be needed for ResNet
        return image[0].to(self.device)  # assumes that you're using GPU

    def show(self, img):
        npimg = img.cpu().detach().numpy()
        npimg = np.transpose(npimg, (1,2,0))
        if npimg.shape[2] == 3:
            plt.imshow(npimg)
        else:
            plt.imshow(npimg[:,:,0], cmap='gray')
            
    def imshow(self, img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.cpu().detach().numpy()
        plt.figure(figsize = (10,2))
        plt.imshow(np.transpose(npimg, (1, 2, 0)), aspect='auto')

    def data_generator(self):
        root = self.data_path
        batch_size = self.batch_size
        
        images_dir = root + 'trainA/'
        labels_dir = root + 'trainB/'

        while True:
            x, y = [], []
            idx = np.random.choice(self.names, batch_size)
            for i in range(idx.shape[0]):
                x.append(self.image_loader(images_dir + idx[i]))
                y.append(self.image_loader(labels_dir + idx[i]))
            yield torch.stack(x), torch.stack(y)
            
data = DataLoader()

In [ ]:
class SegNet(nn.Module):
    def __init__(self,input_nbr,label_nbr):
        super(SegNet, self).__init__()

        batchNorm_momentum = 0.1

        self.conv11 = nn.Conv2d(input_nbr, 64, kernel_size=3, padding=1)
        self.bn11 = nn.BatchNorm2d(64, momentum= batchNorm_momentum)
        self.conv12 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn12 = nn.BatchNorm2d(64, momentum= batchNorm_momentum)

        self.conv21 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn21 = nn.BatchNorm2d(128, momentum= batchNorm_momentum)
        self.conv22 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn22 = nn.BatchNorm2d(128, momentum= batchNorm_momentum)

        self.conv31 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn31 = nn.BatchNorm2d(256, momentum= batchNorm_momentum)
        self.conv32 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn32 = nn.BatchNorm2d(256, momentum= batchNorm_momentum)
        self.conv33 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn33 = nn.BatchNorm2d(256, momentum= batchNorm_momentum)

        self.conv41 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn41 = nn.BatchNorm2d(512, momentum= batchNorm_momentum)
        self.conv42 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn42 = nn.BatchNorm2d(512, momentum= batchNorm_momentum)
        self.conv43 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn43 = nn.BatchNorm2d(512, momentum= batchNorm_momentum)

        self.conv51 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn51 = nn.BatchNorm2d(512, momentum= batchNorm_momentum)
        self.conv52 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn52 = nn.BatchNorm2d(512, momentum= batchNorm_momentum)
        self.conv53 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn53 = nn.BatchNorm2d(512, momentum= batchNorm_momentum)

        self.conv53d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn53d = nn.BatchNorm2d(512, momentum= batchNorm_momentum)
        self.conv52d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn52d = nn.BatchNorm2d(512, momentum= batchNorm_momentum)
        self.conv51d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn51d = nn.BatchNorm2d(512, momentum= batchNorm_momentum)

        self.conv43d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn43d = nn.BatchNorm2d(512, momentum= batchNorm_momentum)
        self.conv42d = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn42d = nn.BatchNorm2d(512, momentum= batchNorm_momentum)
        self.conv41d = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.bn41d = nn.BatchNorm2d(256, momentum= batchNorm_momentum)

        self.conv33d = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn33d = nn.BatchNorm2d(256, momentum= batchNorm_momentum)
        self.conv32d = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.bn32d = nn.BatchNorm2d(256, momentum= batchNorm_momentum)
        self.conv31d = nn.Conv2d(256,  128, kernel_size=3, padding=1)
        self.bn31d = nn.BatchNorm2d(128, momentum= batchNorm_momentum)

        self.conv22d = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn22d = nn.BatchNorm2d(128, momentum= batchNorm_momentum)
        self.conv21d = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.bn21d = nn.BatchNorm2d(64, momentum= batchNorm_momentum)

        self.conv12d = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn12d = nn.BatchNorm2d(64, momentum= batchNorm_momentum)
        self.conv11d = nn.Conv2d(64, label_nbr, kernel_size=3, padding=1)


    def forward(self, x):

        # Stage 1
        x11 = F.relu(self.bn11(self.conv11(x)))
        x12 = F.relu(self.bn12(self.conv12(x11)))
        x1p, id1 = F.max_pool2d(x12,kernel_size=2, stride=2,return_indices=True)

        # Stage 2
        x21 = F.relu(self.bn21(self.conv21(x1p)))
        x22 = F.relu(self.bn22(self.conv22(x21)))
        x2p, id2 = F.max_pool2d(x22,kernel_size=2, stride=2,return_indices=True)

        # Stage 3
        x31 = F.relu(self.bn31(self.conv31(x2p)))
        x32 = F.relu(self.bn32(self.conv32(x31)))
        x33 = F.relu(self.bn33(self.conv33(x32)))
        x3p, id3 = F.max_pool2d(x33,kernel_size=2, stride=2,return_indices=True)

        # Stage 4
        x41 = F.relu(self.bn41(self.conv41(x3p)))
        x42 = F.relu(self.bn42(self.conv42(x41)))
        x43 = F.relu(self.bn43(self.conv43(x42)))
        x4p, id4 = F.max_pool2d(x43,kernel_size=2, stride=2,return_indices=True)

        # Stage 5
        x51 = F.relu(self.bn51(self.conv51(x4p)))
        x52 = F.relu(self.bn52(self.conv52(x51)))
        x53 = F.relu(self.bn53(self.conv53(x52)))
        x5p, id5 = F.max_pool2d(x53,kernel_size=2, stride=2,return_indices=True)


        # Stage 5d
        x5d = F.max_unpool2d(x5p, id5, kernel_size=2, stride=2)
        x53d = F.relu(self.bn53d(self.conv53d(x5d)))
        x52d = F.relu(self.bn52d(self.conv52d(x53d)))
        x51d = F.relu(self.bn51d(self.conv51d(x52d)))

        # Stage 4d
        x4d = F.max_unpool2d(x51d, id4, kernel_size=2, stride=2)
        x43d = F.relu(self.bn43d(self.conv43d(x4d)))
        x42d = F.relu(self.bn42d(self.conv42d(x43d)))
        x41d = F.relu(self.bn41d(self.conv41d(x42d)))

        # Stage 3d
        x3d = F.max_unpool2d(x41d, id3, kernel_size=2, stride=2)
        x33d = F.relu(self.bn33d(self.conv33d(x3d)))
        x32d = F.relu(self.bn32d(self.conv32d(x33d)))
        x31d = F.relu(self.bn31d(self.conv31d(x32d)))

        # Stage 2d
        x2d = F.max_unpool2d(x31d, id2, kernel_size=2, stride=2)
        x22d = F.relu(self.bn22d(self.conv22d(x2d)))
        x21d = F.relu(self.bn21d(self.conv21d(x22d)))

        # Stage 1d
        x1d = F.max_unpool2d(x21d, id1, kernel_size=2, stride=2)
        x12d = F.relu(self.bn12d(self.conv12d(x1d)))
        x11d = self.conv11d(x12d)

        return x11d
    
#     def load_weights(self, model_path):
#         s_dict = self.state_dict()# create a copy of the state dict
#         th = torch.load(model_path).state_dict() # load the weigths
#         # for name in th:
#             # s_dict[corresp_name[name]] = th[name]
#         self.load_state_dict(th)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        
        # Convolution layers
        
        # input is (nc) x 256 x 256
        self.conv1 = nn.Conv2d(nc, ngf, 4, 2, 1, bias=True)
        self.lr1 = nn.LeakyReLU(inplace=True)
        # state size. (ngf) x 128 x 128
        self.conv2 = nn.Conv2d(ngf, ngf*2, 4, 2, 1, bias=True)
        self.bn2 = nn.BatchNorm2d(ngf*2)
        self.lr2 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*2) x 64 x 64
        self.conv3 = nn.Conv2d(ngf*2, ngf*4, 4, 2, 1, bias=True)
        self.bn3 = nn.BatchNorm2d(ngf*4)
        self.lr3 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*4) x 32 x 32
        self.conv4 = nn.Conv2d(ngf*4, ngf*8, 4, 2, 1, bias=True)
        self.bn4 = nn.BatchNorm2d(ngf*8)
        self.lr4 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 16 x 16
        self.conv5 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn5 = nn.BatchNorm2d(ngf*8)
        self.lr5 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 8 x 8
        self.conv6 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn6 = nn.BatchNorm2d(ngf*8)
        self.lr6 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 4 x 4
        self.conv7 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn7 = nn.BatchNorm2d(ngf*8)
        self.lr7 = nn.LeakyReLU(inplace=True)
        # state size. (ngf*8) x 2 x 2
        self.conv8 = nn.Conv2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.bn8 = nn.BatchNorm2d(ngf*8)
        self.r8 = nn.ReLU(inplace=True)
        
        # Transpose Convolutional Layers
        
        # input is (ngf*8) x 1 x 1
        self.tr_conv1 = nn.ConvTranspose2d(ngf*8, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn1 = nn.BatchNorm2d(ngf*8)
        self.tr_r1 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 2 x 2
        self.tr_conv2 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn2 = nn.BatchNorm2d(ngf*8)
        self.tr_r2 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 4 x 4
        self.tr_conv3 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn3 = nn.BatchNorm2d(ngf*8)
        self.tr_r3 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 8 x 8
        self.tr_conv4 = nn.ConvTranspose2d((ngf*8)*2, ngf*8, 4, 2, 1, bias=True)
        self.tr_bn4 = nn.BatchNorm2d(ngf*8)
        self.tr_r4 = nn.ReLU(inplace=True)
        # state size. (ngf*8)*2 x 16 x 16
        self.tr_conv5 = nn.ConvTranspose2d((ngf*8)*2, ngf*4, 4, 2, 1, bias=True)
        self.tr_bn5 = nn.BatchNorm2d(ngf*4)
        self.tr_r5 = nn.ReLU(inplace=True)
        # state size. (ngf*4)*2 x 32 x 32
        self.tr_conv6 = nn.ConvTranspose2d((ngf*4)*2, ngf*2, 4, 2, 1, bias=True)
        self.tr_bn6 = nn.BatchNorm2d(ngf*2)
        self.tr_r6 = nn.ReLU(inplace=True)
        # state size. (ngf*2)*2 x 64 x 64
        self.tr_conv7 = nn.ConvTranspose2d((ngf*2)*2, ngf, 4, 2, 1, bias=True)
        self.tr_bn7 = nn.BatchNorm2d(ngf)
        self.tr_r7 = nn.ReLU(inplace=True)
        # state size. (ngf)*2 x 128 x 128
        self.tr_conv8 = nn.ConvTranspose2d((ngf)*2, nc, 4, 2, 1, bias=True)
        self.out = nn.Tanh()
        # state size. (nc) x 256 x 256
    
    def forward(self, x):
        c1 = self.conv1(x)
        c2 = self.bn2(self.conv2(self.lr1(c1)))
        c3 = self.bn3(self.conv3(self.lr2(c2)))
        c4 = self.bn4(self.conv4(self.lr3(c3)))
        c5 = self.bn5(self.conv5(self.lr4(c4)))
        c6 = self.bn6(self.conv6(self.lr5(c5)))
        c7 = self.bn7(self.conv7(self.lr6(c6)))
        c8 = self.bn8(self.conv8(self.lr7(c7)))
        
        t1 = self.tr_bn1(self.tr_conv1(self.r8(c8)))
        t1 = torch.cat((t1, c7), dim=1)
        t2 = self.tr_bn2(self.tr_conv2(self.tr_r1(t1)))
        t2 = torch.cat((t2, c6), dim=1)
        t3 = self.tr_bn3(self.tr_conv3(self.tr_r2(t2)))
        t3 = torch.cat((t3, c5), dim=1)
        t4 = self.tr_bn4(self.tr_conv4(self.tr_r3(t3)))
        t4 = torch.cat((t4, c4), dim=1)
        t5 = self.tr_bn5(self.tr_conv5(self.tr_r4(t4)))
        t5 = torch.cat((t5, c3), dim=1)
        t6 = self.tr_bn6(self.tr_conv6(self.tr_r5(t5)))
        t6 = torch.cat((t6, c2), dim=1)
        t7 = self.tr_bn7(self.tr_conv7(self.tr_r6(t6)))
        t7 = torch.cat((t7, c1), dim=1)
        t8 = self.tr_conv8(self.tr_r7(t7))
        t8 = self.out(t8)
        return t8
    

In [ ]:
model = Generator().to(device)
# model = SegNet(3,3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002, betas=(0.5, 0.999))
# model.load_state_dict(torch.load("vgg16-00b39a1b.pth"))
# model.load_weights("vgg16-00b39a1b.pth")
# model.to(device)
# Loss = nn.BCELoss()
# Loss = nn.CrossEntropyLoss()
Loss = torch.nn.L1Loss()

In [ ]:
def dice_coef(label, output):
    smooth = 1e-5
    label = torch.round(label.view(-1))
    output = torch.round(output.view(-1))
    isct = torch.sum(torch.mul(label,output))
    return (2 *(isct) / (torch.sum(label) + torch.sum(output)))

In [ ]:
for epoch in range(0, 100):
    for i in range(num_images // batch_size):
        x, y = next(data.data_generator())
        img = Variable(x).to(device)
        label = Variable(y).to(device)
        predicted = model(img[0:12,:])
#         print(predicted.size())
#         print(type(label))
        loss = Loss(predicted,label[0:12,:])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(i%50 == 0):
            val = model(img[12:16,:])
            val_loss = Loss(val,label[12:16,:])
            print ('Epoch: {}, Batch: {}, train_Loss: {}, val_loss: {}, IOU_Train: {}, IOU_Val: {}' 
                   .format(epoch+1, i, loss.item(),val_loss.item(),dice_coef(label[0:12,:],predicted), dice_coef(label[12:16,:],val)))
            img_sample = torch.cat((label[14,:].data, val[2,:].data),-1)
#             print(type(img_sample))
            save_image(img_sample, '/datasets/home/73/673/h6gupta/Project/Sematic_Segmentation/cross_entropy/output/%d.png' % (epoch),nrow=2, normalize=True)
    torch.save(model.state_dict(), '/datasets/home/73/673/h6gupta/Project/Sematic_Segmentation/cross_entropy/Model_Semantic.pth')